In [ ]:
!pip install pytorch_lightning

# IMPORT

In [ ]:
# Model
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer, AdamW
from pytorch_lightning import LightningModule, Trainer

# Dataset
import pandas as pd
import zipfile
import gdown
#from datasets import Dataset, DatasetDict

import os
import io
import traceback

import csv

import numpy as np
import matplotlib.pyplot as plt


**Reproducibility**

In [ ]:
# Set the seed
seed = 46

# Set seed for torch, numpy and random libraries
#torch.backends.cudnn.deterministic = True
#torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
np.random.seed(seed)

# Set the devide mode on GPU (if available CUDA for Nvidia and  MPS for Apple Silicon) or CPU
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

# DATASET

In [ ]:
def download_csv(link_zipped_csv, gdrive_link, zipped_file):

    file_id = link_zipped_csv.split('/')[-2]  # Take the file_id (Ex. "https://drive.google.com/file/d/1BMj4BGXxIMzsd-GYSAEMpB7CF0XB87UT/view?usp=sharing" => file_id: 1BMj4BGXxIMzsd-GYSAEMpB7CF0XB87UT)
    download_link = gdrive_link + file_id # Create the path for gdown (Ex. https://drive.google.com/uc?id={YOUR_FILE_ID})

    try:
        if not os.path.exists(zipped_file):

            gdown.download(
                download_link,
                zipped_file,
                quiet=False
                )
        else:
            print("CSV file already downloaded!")


    except Exception as error:
        print("An error occured:", error)
        traceback.print_exc()

In [ ]:
def unzip_csv(csv_zip, csv_dir):

    try:
        if not os.path.exists(csv_dir):
            os.mkdir(csv_dir) # Create the csv_dir directory where we extract files (if not exists)

        if len(os.listdir(csv_dir)) == 0:
            with zipfile.ZipFile(csv_zip, 'r') as zip:
                filelist = zip.namelist() # list of the file inside zip : ['csv /multilingual_nli_test_df.csv', 'csv /tweet_emotions.csv', ...]

                # Iterate over all file in the zip file to extract them
                for filename in filelist:
                    zip.extract(filename, csv_dir) # Extract the file inside the csv_dir

        else:
            print("CSV file already unzipped!")


    except Exception as error:
        print("An error occured:", error)
        traceback.print_exc()

# Download CSV

We have downloaded different csv files containing text from:

- Kaggle: https://www.kaggle.com/datasets?search=text
- Hugging Face: https://huggingface.co/datasets?modality=modality:text&sort=trending

Then we have zipped all csv and loading on Google Drive the csv.zip file.

In this part there is the downloading of the entire zipped dataset to preprocess (eliminate labels, taking only contents).

In particular:

1. muhammadravi251001/multilingual-nli-dataset: https://huggingface.co/datasets/muhammadravi251001/multilingual-nli-dataset/tree/main

?? 2. Multi-lingual HateSpeech Dataset: https://www.kaggle.com/datasets/wajidhassanmoosa/multilingual-hatespeech-dataset?select=Dataset

3. Multilingual Sentiment Datasets: https://github.com/tyqiangz/multilingual-sentiment-datasets/tree/main

4. molamin/Kinyarwanda_Engligh_Multilingual_ASR: https://huggingface.co/datasets/molamin/Kinyarwanda_Engligh_Multilingual_ASR/tree/main

5. odunola/multilingual-sentiments: https://huggingface.co/datasets/odunola/multilingual-sentiments/tree/main

To simplify the process, we have renamed as "text" each column that we want to take from csv file, discarding the others!


In [ ]:
link_zipped_csv = 'https://drive.google.com/file/d/1WrXbLSkZzS-QGDjWk2_6dLOBIo-4KMBd/view?usp=drive_link'
gdrive_link = 'https://drive.google.com/uc?id='
csv_dir = './csv'
zipped_file = './csv.zip'

download_csv(
    link_zipped_csv,
    gdrive_link,
    zipped_file
)

unzip_csv(
    zipped_file,
    csv_dir,
)


CSV file already downloaded!
CSV file already unzipped!


In [ ]:
def load_dataset_dict(csv_path):

    # Initialization of the dictionary of input data (text) {'filename': ['hi ...','I am ...']}
    text_dict = {}

    filelist = os.listdir(csv_path) # List of all csv file ['filename_1.csv', ...]

    # Iterate over all csv file in the directory
    for filename in filelist:
        # Initialization of the list containing all the texts in a csv file

        text_dict[filename] = []
        # Open the i-th csv file
        with open(csv_path + '/' + filename) as csv_file:

            # Read row by row. Each row is a dictionary with:
            # 1. Key: Name of the column
            # 2. Value: Content of the row
            reader = csv.DictReader(csv_file)

            # Iterate over each row of the csv
            # Ex. ['audio_filepath': 'test_data/37506.wav', 'duration': '15.9279375', 'text': 'maze mvugisha ururimi nti everyone should have..]
            for row in reader:
                # Append to the list each row of the column text
                text_dict[filename].append(row['text'])

    return text_dict

In [ ]:
def texty(text_dict, min=10, max=500):

    # Create a list of the type ["my name is", "Hi, how are you?", ....]
    text_list = []

    text_dict_values = []
    # Put in a single list all the values (rows of all csv files)
    for value in text_dict.values():
        text_dict_values += value

    # Number of total row in all csv files
    total_row = len(text_dict_values)

    #
    sum_ = 0
    random_length_list = []

    while(True):
        random_length = np.random.randint(min,max)
        sum_ += random_length
        random_length_list.append(random_length)
        if sum_ >= total_row - max:
            random_length_list.append(total_row - sum_)
            break

    start = 0
    for length in random_length_list:
        text_list.append('\n'.join(text_dict_values[start:start+length+1]))
        start += length

    return text_list

In [ ]:
def zippy(text_list):

  # Initialization of the dictionary of labels (zip file) {'filename': ['101 ...','010 ...']}
  labels_list = []

  for text in text_list:
    zip_buffer = io.BytesIO()

    with zipfile.ZipFile(file=zip_buffer, mode='w', compression=zipfile.ZIP_LZMA) as byte:
      byte.writestr("file.txt", text)

    labels_list.append(zip_buffer.getvalue().hex())

  return labels_list

In [ ]:
def split(listToSplit, splitPercentages):
  length = len(listToSplit)
  splittedList = []
  lastIndex = 0
  for percentageIndex in range(len(splitPercentages)):
    print(lastIndex)
    splittedList.append(listToSplit[lastIndex:lastIndex+int(splitPercentages[percentageIndex]*length)])
    lastIndex += int(splitPercentages[percentageIndex]*length)

  return splittedList



In [ ]:
class ZipDataset(Dataset):

    def __init__(self, data, labels, model_name, to_tokenize=True):
        self.data = data
        self.labels = labels
        self.to_tokenize = to_tokenize

        if self.to_tokenize:
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.tokenize(data, labels)

    def __getitem__(self, index):
        if self.to_tokenize:
            return self.tokenized_data[index], self.tokenized_labels[index]
        else:
            return self.data[index]['input_ids'], self.data[index]['attention_mask'], self.labels[index]['input_ids']

    def __len__(self):
        return len(self.data)

    def tokenize(self, text_list, labels_list):
        self.tokenized_data = []
        self.tokenized_labels = []
        for text, label in zip(text_list,labels_list):

            tokenized_text = self.tokenizer(
                text,
                padding="max_length",
                truncation=True,
                return_tensors="pt"
            )
            self.tokenized_data.append(tokenized_text)


            tokenized_label = self.tokenizer(
                label,
                padding="max_length",
                truncation=True,
                return_tensors="pt"
            )
            self.tokenized_labels.append(tokenized_label)

In [ ]:
text_dict = load_dataset_dict("./csv")

text_list = texty(text_dict=text_dict)

splitPerc = [0.7,0.2,0.1]


#print(text_list[3])
#print(len(text_list[5]))

labels_list = zippy(text_list)
print(labels_list[4])

...


In [ ]:
splitted_text = split(text_list, splitPerc)

0
144
185


In [ ]:
len(splitted_text[1])

41

In [ ]:
len(text_list)

206

In [ ]:
splitted_labels = split(labels_list, splitPerc)

0
144
185


# MODEL

In [ ]:
model_name = "bert-base-uncased"
dataset_train = ZipDataset(text_list[0], labels_list[0], model_name, to_tokenize=True)
dataset_test = ZipDataset(text_list[1], labels_list[1], model_name, to_tokenize=True)
dataset_val = ZipDataset(text_list[2], labels_list[2], model_name, to_tokenize=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
data, label = dataset_train[0]

print(data)
print(label)

{'input_ids': tensor([[ 101, 1047,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,

In [ ]:
class FineTuneModelSeq2seq(LightningModule):
    def __init__(self, model_name, learning_rate=2e-5):
        super(FineTuneModelSeq2seq, self).__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        self.learning_rate = learning_rate

    def forward(self, input_ids, attention_mask, labels=None):
        return self.model(input_ids, attention_mask=attention_mask, labels=labels)

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        outputs = self(input_ids, attention_mask, labels)
        loss = outputs.loss
        return loss

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.learning_rate)
        return optimizer

In [ ]:
class FineTuneModelCausal(LightningModule):
    def __init__(self, model_name, learning_rate=2e-5):
        super(FineTuneModelCausal, self).__init__()
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.learning_rate = learning_rate

    def forward(self, input_ids, attention_mask, labels=None):
        return self.model(input_ids, attention_mask=attention_mask, labels=labels)

    def training_step(self, batch, batch_idx):
        print(batch)
        input_ids, attention_mask, labels = batch
        outputs = self(input_ids, attention_mask, labels)
        loss = outputs.loss
        return loss

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.learning_rate)
        return optimizer

In [ ]:
dataloader_train = DataLoader(dataset_train)
dataloader_test = DataLoader(dataset_test)
dataloader_val = DataLoader(dataset_val)


In [ ]:
model = FineTuneModelCausal(model_name)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


In [ ]:
trainer = Trainer()

trainer.fit(model, dataloader_train, dataloader_val)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type            | Params | Mode
-------------------------------------------------
0 | model | BertLMHeadModel | 109 M  | eval
-------------------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
438.057   Total estimated model params size (MB)
0         Modules in train mode
233       Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

[{'input_ids': tensor([[[ 101, 1047,  102,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    

ValueError: not enough values to unpack (expected 3, got 2)